In [1]:
import py_stringsimjoin as ssj
from py_stringsimjoin.debugmatcher.debug_matcher import *
from py_stringsimjoin.labeler.labeler import *
from py_stringsimjoin.active_learner.active_learner import *
from sklearn.linear_model import LogisticRegression
import pandas as pd
import py_stringmatching as sm
from py_stringsimjoin.sampler.sample import *
from py_stringsimjoin.feature.autofeaturegen import *
from py_stringsimjoin.feature.extractfeatures import *

In [2]:
ws=sm.WhitespaceTokenizer(return_set=True)
qg3=sm.QgramTokenizer(qval=3,return_set=True)

In [3]:
ldf=pd.read_csv('http://pages.cs.wisc.edu/~anhai/data/corleone_data/citations/dblp.csv')
rdf=pd.read_csv('http://pages.cs.wisc.edu/~anhai/data/corleone_data/citations/google_scholar.csv')

In [5]:
import re
#ldf['cln_title'] = ldf['title'].apply(lambda r: re.sub(r'[^\w\s]',' ',str(r).lower()))
#rdf['cln_title'] = rdf['title'].apply(lambda r: re.sub(r'[^\w\s]',' ',str(r).lower()))
l=[]
for idx, row in ldf.iterrows():
    s=re.sub(r'[^\w\s]',' ',str(row['title']).lower())
    if pd.notnull(row['authors']):
        s += re.sub(r'[^\w\s]',' ',str(row['authors']).lower())
    l.append(s)
r=[]
for idx, row in rdf.iterrows():
    s=re.sub(r'[^\w\s]',' ',str(row['title']).lower())
    if pd.notnull(row['authors']):
        s += re.sub(r'[^\w\s]',' ',str(row['authors']).lower())
    r.append(s)
ldf['cln_title'] = l
rdf['cln_title'] = r

In [6]:
seed=pd.read_csv('/scratch/citseed.csv')

In [7]:
c=sample_pairs(ldf,rdf,'id','id','cln_title','cln_title',100000,25,seed)

0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:00:02


In [8]:
len(c)

100004

In [9]:
labeled_c = label_table_using_gold(c, 'l_id', 'r_id', '/scratch/citgold.csv')
sum(labeled_c['label'])

322

In [10]:
ft=get_features()
ft.drop('dice_ws', inplace=True)
ft.drop('cosine_ws', inplace=True)
ft.drop('overlap_ws', inplace=True)
ft.drop('overlap_coefficient_ws', inplace=True)

In [11]:
ft.ix['jaccard_ws']['tokenizer'] = qg3
fvs = extract_feature_vecs(c, 'l_id','r_id',ldf,rdf,'id','id','cln_title','cln_title',ft,n_jobs=4)

0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:00:25


In [12]:
len(fvs)

100004

In [13]:
lr=LogisticRegression()
al=ActiveLearner(lr,20,20,'/scratch/citgold.csv',seed)

In [14]:
lp = al.learn(fvs, '_id', 'l_id','r_id')

                              l_id          r_id  jaccard_ws
_id                                                         
100000  conf/vldb/ChakrabartiDAR97  8IR1NrQ4zm8J    0.784000
100001  conf/vldb/ChakrabartiDAR97  DKUouEkK_b4J    0.017647
100002   conf/sigmod/LabrinidisR00  DKUouEkK_b4J    1.000000
100003   conf/sigmod/LabrinidisR00  8IR1NrQ4zm8J    0.019608
                              l_id          r_id  jaccard_ws  label
_id                                                                
100000  conf/vldb/ChakrabartiDAR97  8IR1NrQ4zm8J    0.784000      1
100001  conf/vldb/ChakrabartiDAR97  DKUouEkK_b4J    0.017647      0
100002   conf/sigmod/LabrinidisR00  DKUouEkK_b4J    1.000000      1
100003   conf/sigmod/LabrinidisR00  8IR1NrQ4zm8J    0.019608      0
Selecting next batch...
computing entropy
sorting
sampling
[0.69314718051669544, 0.69314718051669544, 0.69314718025348876, 0.69314717943250836, 0.69314717943250836, 0.69314717943250836, 0.69314717777703194, 0.69314717777703194

py_stringsimjoin/active_learner/active_learner.py:121: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  to_be_labeled_pairs['label'] = labels


In [15]:
lp.head()

,l_id,r_id,jaccard_ws,label
_id,,,,
100000,conf/vldb/ChakrabartiDAR97,8IR1NrQ4zm8J,0.784000,1
100001,conf/vldb/ChakrabartiDAR97,DKUouEkK_b4J,0.017647,0
100002,conf/sigmod/LabrinidisR00,DKUouEkK_b4J,1.000000,1
100003,conf/sigmod/LabrinidisR00,8IR1NrQ4zm8J,0.019608,0
19323,conf/sigmod/SchusterW01,nPkMLSiC3JQJ,0.228070,0


In [16]:
lp.reset_index(level=0, inplace=True)
sum(lp['label'])

172

In [17]:
import py_entitymatching as em
em.set_property(lp,'key','_id')
em.set_property(ldf,'key','id')
em.set_property(rdf,'key','id')
em.set_ltable(lp,ldf)
em.set_rtable(lp,rdf)
em.set_fk_ltable(lp,'l_id')
em.set_fk_rtable(lp,'r_id')
splits = em.split_train_test(lp)
P=splits['train']
Q=splits['test']

In [18]:
len(P), len(Q)
sum(P['label']), sum(Q['label'])

(80, 92)

In [19]:
lr1=LogisticRegression()
dbg = debug_matcher(lr1, P, Q, 'jaccard_ws', 'label', 'l_id', 'r_id', ldf, rdf, 'id', 'id')

In [52]:
dbg['false_neg'][['l_title', 'r_title', 'jaccard_ws']]

,l_title,r_title,jaccard_ws
225,Fast Computation of Sparse Datacubes,Fast Computations of Sparse Cubes,0.585714
60,Prospector: A Content-Based Multimedia Server for Massively Parallel Architectures,"etal., Prospector: A Content-Based Multimedia Object Server for Massively Parallel Architectures",0.510638
59,Specification and Implementation of Exceptions in Workflow Management Systems,Specification and Implementation of Exceptions in Workflow Management Systems [J],0.571429
66,"Title, Preface to the Special Issue on Persistent Object Systems",Preface to special issue on persistent object systems,0.544444
62,The Database Research Group at ETH Zurich,From the KERNEL to the COSMOS: the database research group at ETH Zurich,0.474576
281,TelegraphCQ: Continuous Dataflow Processing,AD 2003. TelegraphCQ: Continuous Dataflow Processing for an Uncertain World,0.361702
106,Energy Efficient Indexing on Air,Indexing on air,0.607595
103,Data Stream Query Processing: A Tutorial,Data Stream Query Processing: A Tutorial,0.606061
130,LH*RS: A High-Availability Scalable Distributed Data Structure using Reed Solomon Codes,LH*s: A High-Availability and High-Security Scalable Distributed Data Structure,0.496124
68,Object Fusion in Mediator Systems,Object fusion in mediator systems (extended version). Available by anonymous ftp at db,0.507463


In [21]:
dbg['stats']

{'f1': 0.7560975609756099,
 'precision': 0.8611111111111112,
 'recall': 0.6739130434782609}

In [22]:
-lr1.intercept_/lr1.coef_

array([[ 0.61488554]])

In [42]:
len(dbg['false_neg']), len(dbg['false_pos'])

(19, 7)

In [23]:
lr2=LogisticRegression()
lr2.fit(lp[['jaccard_ws']].values, lp['label'].values)
-lr2.intercept_/lr2.coef_

array([[ 0.5777797]])

In [59]:
from sklearn.svm import LinearSVC
lsvc = LinearSVC()
lsvc.fit(lp[['jaccard_ws']].values, lp['label'].values)
-lsvc.intercept_/lsvc.coef_

array([[ 0.56236397]])

In [65]:
what_if_matrix(lp, 'jaccard_ws', 'label', 0.57, 0.02, 10)

,threshold,precision,recall,f1
0,0.37,0.557377,0.988372,0.712788
1,0.39,0.599291,0.982558,0.744493
2,0.41,0.639847,0.970930,0.771363
3,0.43,0.670683,0.970930,0.793349
4,0.45,0.694561,0.965116,0.807786
5,0.47,0.723684,0.959302,0.825000
6,0.49,0.752336,0.936047,0.834197
7,0.51,0.791878,0.906977,0.845528
8,0.53,0.812834,0.883721,0.846797
9,0.55,0.808989,0.837209,0.822857


In [29]:
lp1=lp[lp['jaccard_ws'] >= 0.34]
len(lp1)

376

In [30]:
lp1.head()

,_id,l_id,r_id,jaccard_ws,label
0,100000,conf/vldb/ChakrabartiDAR97,8IR1NrQ4zm8J,0.784000,1
2,100002,conf/sigmod/LabrinidisR00,DKUouEkK_b4J,1.000000,1
24,37211,journals/sigmod/EnglertGH95,JRXuKm3E_foJ,1.000000,1
25,5067,journals/sigmod/NobleS95,rRvcD6HKhasJ,0.941176,1
26,61027,journals/tods/Chomicki03,pHuL-9kHWH0J,1.000000,1


In [51]:
cc=ssj.jaccard_join(ldf,rdf,'id','id','cln_title','cln_title',qg3,0.5777797,l_out_attrs=['year'], r_out_attrs=['year'],n_jobs=4)

0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:00:39


In [53]:
len(cc)

5070

In [54]:
labeled_cc = label_table_using_gold(cc, 'l_id', 'r_id', '/scratch/citgold.csv')
sum(labeled_cc['label'])


4626

In [55]:
4626/5070.0, 4626/5347.0

(0.9124260355029585, 0.8651580325416122)

In [56]:
cc1=ssj.jaccard_join(ldf,rdf,'id','id','cln_title','cln_title',qg3,0.34,l_out_attrs=['year'], r_out_attrs=['year'],n_jobs=4)

0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:02:00


In [57]:
labeled_cc1 = label_table_using_gold(cc1, 'l_id', 'r_id', '/scratch/citgold.csv')
sum(labeled_cc1['label'])

5270

In [60]:
cc2=ssj.jaccard_join(ldf,rdf,'id','id','cln_title','cln_title',qg3,0.56236397,n_jobs=4)


0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:00:43


In [61]:
labeled_cc2 = label_table_using_gold(cc2, 'l_id', 'r_id', '/scratch/citgold.csv')
sum(labeled_cc2['label'])

4705

In [62]:
len(cc2)

5247

In [64]:
4705/5247.0, 4705/5347.0

(0.8967028778349533, 0.8799326725266504)